In [40]:
import pandas as pd
# import pandana as pdna
import time
import numpy as np
import matplotlib.pyplot as plt
import os

pd.options.display.max_columns = 80

In [41]:
bay_counties = [1, 13, 97, 95, 81, 85, 75, 41, 55]

In [42]:
os.listdir('/home/data/CHTS_csv_format/data')

['Deliv_HH.csv',
 'Deliv_ACTIVITY.csv',
 'Deliv_LD.csv',
 'LookUp_LD.csv',
 'LookUp_Home.csv',
 'ASSN_TravelDate.csv',
 'LookUp_PLACE.csv',
 'LookUp_PER.csv',
 'Deliv_PER.csv',
 'Deliv_VEH.csv',
 'Deliv_PLACE.csv']

In [43]:
place = pd.read_csv('/home/data/CHTS_csv_format/data/Deliv_PLACE.csv')
person = pd.read_csv('/home/data/CHTS_csv_format/data/Deliv_PER.csv')
activity = pd.read_csv('/home/data/CHTS_csv_format/data/Deliv_ACTIVITY.csv')
schools = pd.read_csv('/home/juan/ual_model_workspace/spring-2019-models/notebooks-juan/schools.csv').rename({"parcel_id": "school_parcel_id"}, axis=1)
students = pd.read_csv('/home/juan/ual_model_workspace/spring-2019-models/notebooks-juan/students_with_school_id.csv')


/home/amelia/enter/envs/template-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (22,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/amelia/enter/envs/template-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (40,60,93,97,104) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### In Place table: create unique record identifiers by person, place and hh and create columns for start and end times by combining hour and minute info

In [45]:
place['HHPER'] = place['SAMPN'].map(str) + place['PERNO'].map(str) #sampn is hh identifier, perno is the person num within
#the household

place['HHPERPLA'] = place['HHPER'].map(str) + place['PLANO'].map(str)

# place.loc[ (place['Start_time'] > 24), 'Start_time'] = place['Start_time'] - 24

# place.loc[ (place['End_time'] > 24),'End_time'] = place['End_time'] - 24
place1 = place[['SAMPN', 'HHPER','HHPERPLA','MODE',"ARR_HR", "ARR_MIN", "DEP_HR", "DEP_MIN",'PNAME']]
place1.head()

,SAMPN,HHPER,HHPERPLA,MODE,ARR_HR,ARR_MIN,DEP_HR,DEP_MIN,PNAME
0,1031985,10319851,103198511,NaN,3,0,8,40,HOME
1,1031985,10319851,103198512,6.0,9,2,12,53,BENICIA SENIOR CITIZEN CENTER
2,1031985,10319851,103198513,6.0,13,13,2,59,HOME
3,1031985,10319852,103198521,NaN,3,0,9,16,HOME
4,1031985,10319852,103198522,5.0,9,26,10,43,LOMA VISTA ELEMENTARY SCHOOL


#### In activity table: create unique record identifiers by person, place and hh id

In [46]:
activity['HHPER'] = activity['SAMPN'].map(str) + activity['PERNO'].map(str)
activity['HHPERPLA'] = activity['HHPER'].map(str) + activity['PLANO'].map(str)
activity['count'] = activity.groupby('HHPERPLA').cumcount() +1 #group by unique person-hh-place key, to account 
#for the same person going to the same place but doing different activities, and therefore having different
#trip purposes. cumcount() +1 numbers these different activities for the same person-place trip
activity2 = activity.pivot(index='HHPERPLA', columns='count', values='APURP').reset_index()
# #then take these person-place grouped entries and articulate the different activity purposes (up to 3)
activity2.columns = ['HHPERPLA', 'APURP1', 'APURP2', 'APURP3']

activity.head()

,SAMPN,PERNO,PLANO,ACTNO,ACTOTH,ACTHH,ACTOR,ACTWK,ACTSC,ACTRG,ACTFR,ACTOT,APURP,O_APURP,STIME,ETIME,PERWGT,EXPPERWGT,TRIPNO,TCF,TCFPERWGT,EXPTCFPERWGT,HHPER,HHPERPLA,count
0,1031985,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,03:00,07:00,0.052086,17.647568,NaN,NaN,NaN,NaN,10319851,103198511,1
1,1031985,1,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,07:00,08:10,0.052086,17.647568,NaN,NaN,NaN,NaN,10319851,103198511,2
2,1031985,1,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,08:10,08:40,0.052086,17.647568,NaN,NaN,NaN,NaN,10319851,103198511,3
3,1031985,1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37,NaN,09:02,12:48,0.052086,17.647568,1.0,0.969789,0.050512,17.114409,10319851,103198512,1
4,1031985,1,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,13:13,13:38,0.052086,17.647568,2.0,1.011905,0.052706,17.857658,10319851,103198513,1


Classify trip purpose as a school trip or not- if at least one of the 3 trip purposes is for school/studying (5), 
school/classroom/laboratory (17), meals at school/college (18), and also After-school or non-class-related
sports/physical activities (19),  All other after-school or non-class related activities (library, music rehearsal, clubs, etc.) (20)

In [47]:
#place column is "school" if at least one of the 3 trip purpose columns is 5 (school/study trip purpose)
activity2['place'] = np.where(((activity2["APURP1"] == 5.0) | (activity2["APURP1"] == 17.0) |(activity2["APURP1"] == 18.0) |(activity2["APURP1"] == 19.0) |(activity2["APURP1"] == 20.0)
                              |(activity2["APURP2"] == 5.0) | (activity2["APURP2"] == 17.0) |(activity2["APURP2"] == 18.0) |(activity2["APURP2"] == 19.0) |(activity2["APURP2"] == 20.0)
                              |(activity2["APURP3"] == 5.0) | (activity2["APURP3"] == 17.0) |(activity2["APURP3"] == 18.0) |(activity2["APURP3"] == 19.0) |(activity2["APURP3"] == 20.0)
                                                                                             ), "school", "neither")
activity2.head()

,HHPERPLA,APURP1,APURP2,APURP3,place
0,103198511,1.0,2.0,1.0,neither
1,103198512,37.0,NaN,NaN,neither
2,103198513,2.0,7.0,8.0,neither
3,103198521,1.0,2.0,7.0,neither
4,103198522,33.0,NaN,NaN,neither


Classify trip destination as home or not

In [48]:
place2 = pd.merge(place1, activity2, on='HHPERPLA')

place2.loc[place2.PNAME == "HOME", 'place'] = "home"
place2.head()

,SAMPN,HHPER,HHPERPLA,MODE,ARR_HR,ARR_MIN,DEP_HR,DEP_MIN,PNAME,APURP1,APURP2,APURP3,place
0,1031985,10319851,103198511,NaN,3,0,8,40,HOME,1.0,2.0,1.0,home
1,1031985,10319851,103198512,6.0,9,2,12,53,BENICIA SENIOR CITIZEN CENTER,37.0,NaN,NaN,neither
2,1031985,10319851,103198513,6.0,13,13,2,59,HOME,2.0,7.0,8.0,home
3,1031985,10319852,103198521,NaN,3,0,9,16,HOME,1.0,2.0,7.0,home
4,1031985,10319852,103198522,5.0,9,26,10,43,LOMA VISTA ELEMENTARY SCHOOL,33.0,NaN,NaN,neither


Filter for just the hhper that are students and merge with the students dataframe to get the school that they attend

In [49]:
od = pd.read_csv("/home/data/fall_2018/chts_persons_w_zone_ids.csv")
#create unique hh-person-place identifiers from chts od pair data
od['HHPER'] = od['SAMPN'].map(str) + od['PERNO'].map(str)
od = od[['HHPER','zone_id_home','HXCORD','HYCORD']]
od.head()

/home/amelia/enter/envs/template-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (42,95,99,106,152,164) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,HHPER,zone_id_home,HXCORD,HYCORD
0,10319851,1247,-122.182785,38.161600
1,10319852,1247,-122.182785,38.161600
2,10339441,168,-122.470070,37.718091
3,10352741,424,-122.017274,37.388281
4,10352742,424,-122.017274,37.388281


In [59]:
#sampn is hh identifier, perno is the person num within the hh, sp HHPER is a unique person identifier

students['HHPER'] = students['SAMPN'].map(str) + students['PERNO'].map(str) 

#students dataframe- this is just the students in the chts with information about what school they go to
students1 = students[["SAMPN", "HHPER","PERNO","SMODE", 'parcel_id_home', 'zone_id_home', 'school_id']]

students1.head()

print("unique hhper ids for students in place2 df: ", len(students1)) 
print("non unique hh per ids in place2 df: ", len(place2)) 
print("unique hh per ids in place2 df: ", len(place2["HHPER"].unique())) 

unique hhper ids for students in place2 df:  3385
non unique hh per ids in place2 df:  460520
unique hh per ids in place2 df:  108778


#### Assign locations to home and school locations

In [60]:
school_home_locations = pd.merge(place2, students1, on='HHPER')[["SAMPN_x", "HHPER", 'HHPERPLA', 'MODE',
                    'PNAME', 'APURP1', 'APURP2', 'APURP3', 'place', 'PERNO', 'SMODE',
       'parcel_id_home', 'zone_id_home','school_id']]
school_home_locations.head()

print("non unique hhper ids of students matched with schools:", len(school_home_locations))
print("unique hhper ids of students matched with schools: ", len(school_home_locations["HHPER"].unique()))

non unique hhper ids of students matched with schools: 14491
unique hhper ids of students matched with schools:  3362


#### Filter out irregular trips and/or repeats or non work-home trips

In [61]:
#selecting trips by people who go to school at least once

place3 = school_home_locations.groupby('HHPER').filter(lambda x : x['place'].isin(["school"]).any())
place3.head()



,SAMPN_x,HHPER,HHPERPLA,MODE,PNAME,APURP1,APURP2,APURP3,place,PERNO,SMODE,parcel_id_home,zone_id_home,school_id
2,1050385,10503854,105038541,NaN,HOME,1.0,NaN,NaN,home,4,6.0,1257253,311,3279.0
3,1050385,10503854,105038542,6.0,ST CHARLES CHURCH,17.0,NaN,NaN,school,4,6.0,1257253,311,3279.0
4,1050385,10503854,105038543,6.0,HOME,1.0,NaN,NaN,home,4,6.0,1257253,311,3279.0
5,1051388,10513883,105138831,NaN,HOME,1.0,5.0,2.0,home,3,6.0,913377,89,3176.0
6,1051388,10513883,105138832,6.0,PRESIDIO HILL SCHOOL,17.0,18.0,NaN,school,3,6.0,913377,89,3176.0


In [65]:
def count_place(df):
    '''adds a column with each of the home and school trips numbered. ie: the first home trip by that person is 1, 
    the second home trip is 2, etc.'''
    return df.join(df['place'].str.get_dummies()
                  .cumsum().add_suffix('_count')).head(n=15)

trips = place4.groupby('HHPER', group_keys=False).apply(count_place)

trips.head()

/home/amelia/enter/envs/template-env/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:927: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  result = concat(values, axis=self.axis)


,APURP1,APURP2,APURP3,HHPER,HHPERPLA,MODE,PERNO,PNAME,SAMPN_x,SMODE,act_end_time,act_start_time,home_count,neither_count,parcel_id_home,place,school_count,school_id,zone_id_home
2,1.0,NaN,NaN,10503854,105038541,NaN,4,HOME,1050385,6.0,7.500000,3.000000,1.0,NaN,1257253,home,0,3279.0,311
3,17.0,NaN,NaN,10503854,105038542,6.0,4,ST CHARLES CHURCH,1050385,6.0,15.000000,7.833333,1.0,NaN,1257253,school,1,3279.0,311
4,1.0,NaN,NaN,10503854,105038543,6.0,4,HOME,1050385,6.0,2.983333,15.333333,2.0,NaN,1257253,home,1,3279.0,311
5,1.0,5.0,2.0,10513883,105138831,NaN,3,HOME,1051388,6.0,8.016667,3.000000,1.0,NaN,913377,home,0,3176.0,89
6,17.0,18.0,NaN,10513883,105138832,6.0,3,PRESIDIO HILL SCHOOL,1051388,6.0,15.333333,8.250000,1.0,NaN,913377,school,1,3176.0,89


In [99]:
#eliminate records for persons that are never home

neverhome = trips[trips.home_count.isnull()]
neverathome = neverhome["HHPER"].unique()
trips1 = trips[~trips.HHPER.isin(neverathome)] 


# create a trip id for each trip that is 2 numbers where the first is the number of the school trip and the sceond
# is the number of the home trip

trips1["trip_ID"] = trips1["school_count"].map(str) + trips1["home_count"].map(str)
trips1["HHPERTRIP"] = trips1["HHPER"].map(str) + (trips1["trip_ID"]).map(str)
trips1["HHPERTRIP"] = trips1["HHPERTRIP"].map(float)/100
trips1['num_trips'] = trips1.groupby('HHPERTRIP').cumcount() + 1 #adds a column with the cumulative count of trips taken
#by that person that day. 12 is the max.
trips1.head()


/home/amelia/enter/envs/template-env/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/home/amelia/enter/envs/template-env/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/home/amelia/enter/envs/template-env/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

,APURP1,APURP2,APURP3,HHPER,HHPERPLA,MODE,PERNO,PNAME,SAMPN_x,SMODE,act_end_time,act_start_time,home_count,neither_count,parcel_id_home,place,school_count,school_id,zone_id_home,trip_ID,HHPERTRIP,num_trips
2,1.0,NaN,NaN,10503854,105038541,NaN,4,HOME,1050385,6.0,7.500000,3.000000,1.0,NaN,1257253,home,0,3279.0,311,01.0,10503854.01,1
3,17.0,NaN,NaN,10503854,105038542,6.0,4,ST CHARLES CHURCH,1050385,6.0,15.000000,7.833333,1.0,NaN,1257253,school,1,3279.0,311,11.0,10503854.11,1
4,1.0,NaN,NaN,10503854,105038543,6.0,4,HOME,1050385,6.0,2.983333,15.333333,2.0,NaN,1257253,home,1,3279.0,311,12.0,10503854.12,1
5,1.0,5.0,2.0,10513883,105138831,NaN,3,HOME,1051388,6.0,8.016667,3.000000,1.0,NaN,913377,home,0,3176.0,89,01.0,10513883.01,1
6,17.0,18.0,NaN,10513883,105138832,6.0,3,PRESIDIO HILL SCHOOL,1051388,6.0,15.333333,8.250000,1.0,NaN,913377,school,1,3176.0,89,11.0,10513883.11,1


In [106]:
#isolate just the trips that are the start of home-school or school-home trip (place column is the start of the trip)
home_school_trips = trips1.loc[trips['place'].isin(["home","school"])]
home_school_trips.head()

,APURP1,APURP2,APURP3,HHPER,HHPERPLA,MODE,PERNO,PNAME,SAMPN_x,SMODE,act_end_time,act_start_time,home_count,neither_count,parcel_id_home,place,school_count,school_id,zone_id_home,trip_ID,HHPERTRIP,num_trips
2,1.0,NaN,NaN,10503854,105038541,NaN,4,HOME,1050385,6.0,7.500000,3.000000,1.0,NaN,1257253,home,0,3279.0,311,01.0,10503854.01,1
3,17.0,NaN,NaN,10503854,105038542,6.0,4,ST CHARLES CHURCH,1050385,6.0,15.000000,7.833333,1.0,NaN,1257253,school,1,3279.0,311,11.0,10503854.11,1
4,1.0,NaN,NaN,10503854,105038543,6.0,4,HOME,1050385,6.0,2.983333,15.333333,2.0,NaN,1257253,home,1,3279.0,311,12.0,10503854.12,1
5,1.0,5.0,2.0,10513883,105138831,NaN,3,HOME,1051388,6.0,8.016667,3.000000,1.0,NaN,913377,home,0,3176.0,89,01.0,10513883.01,1
6,17.0,18.0,NaN,10513883,105138832,6.0,3,PRESIDIO HILL SCHOOL,1051388,6.0,15.333333,8.250000,1.0,NaN,913377,school,1,3176.0,89,11.0,10513883.11,1


Take just the trips where the place is not the same as the one before it and/or the person is not the same as the one before it. Either it is a different person or the destiation of the trip is different than the destination of the previous trip by the same person.


This captures 'from home' trips that end at work and 'from work' trips that end at home. Also, captures end-of-day 'from home' trips that don't end at work and 'from work' trips that don't end at home (for now, these will be filtered later).


In [120]:
trips_clean = home_school_trips.loc[(home_school_trips.HHPER != home_school_trips.HHPER.shift(-1)) | (home_school_trips.place != home_school_trips.place.shift(-1))]

trips_clean.head()


,APURP1,APURP2,APURP3,HHPER,HHPERPLA,MODE,PERNO,PNAME,SAMPN_x,to_mode,act_end_time,act_start_time,home_count,neither_count,parcel_id_home,place,school_count,school_id,zone_id_home,trip_ID,HHPERTRIP,num_trips,mode_from
2,1.0,NaN,NaN,10503854,105038541,NaN,4,HOME,1050385,6.0,7.500000,3.000000,1.0,NaN,1257253,home,0,3279.0,311,01.0,10503854.01,1,6.0
3,17.0,NaN,NaN,10503854,105038542,6.0,4,ST CHARLES CHURCH,1050385,6.0,15.000000,7.833333,1.0,NaN,1257253,school,1,3279.0,311,11.0,10503854.11,1,6.0
4,1.0,NaN,NaN,10503854,105038543,6.0,4,HOME,1050385,6.0,2.983333,15.333333,2.0,NaN,1257253,home,1,3279.0,311,12.0,10503854.12,1,NaN
5,1.0,5.0,2.0,10513883,105138831,NaN,3,HOME,1051388,6.0,8.016667,3.000000,1.0,NaN,913377,home,0,3176.0,89,01.0,10513883.01,1,6.0
6,17.0,18.0,NaN,10513883,105138832,6.0,3,PRESIDIO HILL SCHOOL,1051388,6.0,15.333333,8.250000,1.0,NaN,913377,school,1,3176.0,89,11.0,10513883.11,1,6.0


Add a column for from mode and rename mode column to specify its the "to-mode"

In [121]:
home_school_trips['mode_from'] = home_school_trips.groupby('HHPER', group_keys=False).MODE.shift(-1)
home_school_trips.rename({"SMODE": "to_mode"}, inplace=True, axis=1)
home_school_trips.head()


/home/amelia/enter/envs/template-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,APURP1,APURP2,APURP3,HHPER,HHPERPLA,MODE,PERNO,PNAME,SAMPN_x,to_mode,act_end_time,act_start_time,home_count,neither_count,parcel_id_home,place,school_count,school_id,zone_id_home,trip_ID,HHPERTRIP,num_trips,mode_from
2,1.0,NaN,NaN,10503854,105038541,NaN,4,HOME,1050385,6.0,7.500000,3.000000,1.0,NaN,1257253,home,0,3279.0,311,01.0,10503854.01,1,6.0
3,17.0,NaN,NaN,10503854,105038542,6.0,4,ST CHARLES CHURCH,1050385,6.0,15.000000,7.833333,1.0,NaN,1257253,school,1,3279.0,311,11.0,10503854.11,1,6.0
4,1.0,NaN,NaN,10503854,105038543,6.0,4,HOME,1050385,6.0,2.983333,15.333333,2.0,NaN,1257253,home,1,3279.0,311,12.0,10503854.12,1,NaN
5,1.0,5.0,2.0,10513883,105138831,NaN,3,HOME,1051388,6.0,8.016667,3.000000,1.0,NaN,913377,home,0,3176.0,89,01.0,10513883.01,1,6.0
6,17.0,18.0,NaN,10513883,105138832,6.0,3,PRESIDIO HILL SCHOOL,1051388,6.0,15.333333,8.250000,1.0,NaN,913377,school,1,3176.0,89,11.0,10513883.11,1,6.0


Get mode dummy columns

In [122]:
hs_trips_mode = pd.get_dummies(home_school_trips, columns=["mode_from"])
hs_trips_mode.head()

,APURP1,APURP2,APURP3,HHPER,HHPERPLA,MODE,PERNO,PNAME,SAMPN_x,to_mode,act_end_time,act_start_time,home_count,neither_count,parcel_id_home,place,school_count,school_id,zone_id_home,trip_ID,HHPERTRIP,num_trips,mode_from_1.0,mode_from_2.0,mode_from_4.0,mode_from_5.0,mode_from_6.0,mode_from_7.0,mode_from_8.0,mode_from_9.0,mode_from_10.0,mode_from_11.0,mode_from_14.0,mode_from_18.0
2,1.0,NaN,NaN,10503854,105038541,NaN,4,HOME,1050385,6.0,7.500000,3.000000,1.0,NaN,1257253,home,0,3279.0,311,01.0,10503854.01,1,0,0,0,0,1,0,0,0,0,0,0,0
3,17.0,NaN,NaN,10503854,105038542,6.0,4,ST CHARLES CHURCH,1050385,6.0,15.000000,7.833333,1.0,NaN,1257253,school,1,3279.0,311,11.0,10503854.11,1,0,0,0,0,1,0,0,0,0,0,0,0
4,1.0,NaN,NaN,10503854,105038543,6.0,4,HOME,1050385,6.0,2.983333,15.333333,2.0,NaN,1257253,home,1,3279.0,311,12.0,10503854.12,1,0,0,0,0,0,0,0,0,0,0,0,0
5,1.0,5.0,2.0,10513883,105138831,NaN,3,HOME,1051388,6.0,8.016667,3.000000,1.0,NaN,913377,home,0,3176.0,89,01.0,10513883.01,1,0,0,0,0,1,0,0,0,0,0,0,0
6,17.0,18.0,NaN,10513883,105138832,6.0,3,PRESIDIO HILL SCHOOL,1051388,6.0,15.333333,8.250000,1.0,NaN,913377,school,1,3176.0,89,11.0,10513883.11,1,0,0,0,0,1,0,0,0,0,0,0,0


### Add columns with dwell time at the origin and travel time to arrival location.

In [ ]:
place['act_start_time'] = place['ARR_HR'] + (place['ARR_MIN']/60)

place['act_end_time'] = place['DEP_HR'] + (place['DEP_MIN']/60)


## Step 4. Assign modes: drive alone, shared, bike, walk, walkTransitwalk, driveTransitwalk, walkTransitdrive.

In [30]:
tripsR['MODE'] = np.where((((tripsR['MODE1'].isin([5.0,6.0,7.0,8.0,9.0,10.0])) |
                                                           (tripsR['MODE2'].isin([5.0,6.0,7.0,8.0,9.0,10.0])) |
                                                           (tripsR['MODE3'].isin([5.0,6.0,7.0,8.0,9.0,10.0])) |
                                                           (tripsR['MODE4'].isin([5.0,6.0,7.0,8.0,9.0,10.0])) |
                                                           (tripsR['MODE5'].isin([5.0,6.0,7.0,8.0,9.0,10.0])) |
                                                           (tripsR['MODE6'].isin([5.0,6.0,7.0,8.0,9.0,10.0])) |
                                                           (tripsR['MODE7'].isin([5.0,6.0,7.0,8.0,9.0,10.0])) |
                                                           (tripsR['MODE8'].isin([5.0,6.0,7.0,8.0,9.0,10.0])) |
                                                           (tripsR['MODE9'].isin([5.0,6.0,7.0,8.0,9.0,10.0])) |
                                                           (tripsR['MODE10'].isin([5.0,6.0,7.0,8.0,9.0,10.0])) |
                                                           (tripsR['MODE11'].isin([5.0,6.0,7.0,8.0,9.0,10.0])) |
                                                           (tripsR['MODE12'].isin([5.0,6.0,7.0,8.0,9.0,10.0])) |
                                                           (tripsR['MODE13'].isin([5.0,6.0,7.0,8.0,9.0,10.0])) |
                                                           (tripsR['MODE14'].isin([5.0,6.0,7.0,8.0,9.0,10.0]))) &
                            ((tripsR['MODE1'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])) |
                             (tripsR['MODE2'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])) |
                             (tripsR['MODE3'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])) |
                             (tripsR['MODE4'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])) |
                             (tripsR['MODE5'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])) |
                             (tripsR['MODE6'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])) |
                             (tripsR['MODE7'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])) |
                             (tripsR['MODE8'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])) |
                             (tripsR['MODE9'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])) |
                             (tripsR['MODE10'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])) |
                             (tripsR['MODE11'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])) |
                             (tripsR['MODE12'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])) |
                             (tripsR['MODE13'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])) |
                             (tripsR['MODE14'].isin([15.0,16.0,17.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0])))),
                          'drive_transit','no')

NameError: name 'tripsR' is not defined